# Modeling Theta: Muti-population recurrent network (with BMTK BioNet) 

Here we will create a heterogenous yet relatively small network consisting of hundreds of cells recurrently connected. All cells will belong to one of four "cell-types". Two of these cell types will be biophysically detailed cells, i.e. containing a full morphology and somatic and dendritic channels and receptors. The other two will be point-neuron models, which lack a full morphology or channels but still act to provide inhibitory and excitory dynamics.

As input to drive the simulation, we will also create an external network of "virtual cells" that synapse directly onto our internal cells and provide spike trains stimulus

**Note** - scripts and files for running this tutorial can be found in the directory [theta](https://github.com/cyneuro/theta)

requirements:
* bmtk
* NEURON 7.4+

## 1. Building the network

#### cells

This network will loosely resemble the rodent hippocampal CA3 region. Along the center of the column will be a population of 50 biophysically detailed neurons: 40 excitatory Scnn1a cells and 10 inhibitory PV cells.

In [3]:
import numpy as np

from bmtk.builder.networks import NetworkBuilder

ExcTotal = 200 # number of RTM excitatory cells
InhTotal = 50  # number WB inhibitory cells
    

net = NetworkBuilder('network')

net.add_nodes(N=CA3eTotal, pop_name='Exc',
        model_type='biophysical',
        model_template='hoc:RTMExcCell')

net.add_nodes(N=CA3oTotal, pop_name='Inh',
        model_type='biophysical',
        model_template='hoc:WBInhCell')


To set the position and rotation of each cell, we use the built in function positions_columinar and xiter_random, which returns a list of values given the parameters. A user could set the values themselves using a list (or function that returns a list) of size N. The parameters like location, ei (potential), params_file, etc. are cell-type parameters, and will be used for all N cells of that type.

The excitory cells are also given a tuning_angle parameter. An instrinsic "tuning angle" is a property found in some cells in the visual cortex. In this model, we will use this property to determine number of strenght of connections between subsets of cells by using custom functions. But in general most models will not have or use a tuning angle, but they may require some other parameter. In general, users can assign whatever custom parameters they want to cells and cell-types and use them as properties for creating connections and running simulations.

Next we continue to create our point (integrate-and-fire) neurons. Notice they don't have properities like y/z rotation or morphology, as they wouldn't apply to point neurons.

In [4]:
from bmtk.simulator.bionet.pyfunction_cache import add_synapse_model
from neuron import h
import random


def load_synapses():
    return

load_synapses()

#### connections

Now we want to create connections between the cells. Depending on the model type, and whether or not the presynpatic "source" cell is excitory or inhibitory, we will have different synpatic model and parameters. Using the source and target filter parameters, we can create different connection types.

To determine excitory-to-excitory connection matrix we want to use distance and tuning_angle property. To do this we create a customized function "dist_tuning_connector"

In [4]:
import random
import math

# list of all synapses created - used for recurrent connections
syn_list = []

###########################################################
# Build custom connection rules
###########################################################
#See bmtk.builder.auxi.edge_connectors
def hipp_dist_connector(source, target, con_pattern, ratio=1, gaussa=0, min_syn=1, max_syn=1):
    """
    :returns: number of synapses per connection
    """

    ratio = float(ratio)
    gaussa = float(gaussa)

    Lamellar = "0"
    Homogenous = "1"
    AxonalPlexus = "2"
    IntPyrFeedback = "3"

    x_ind,y_ind = 0,1
    dx = target['positions'][x_ind] - source['positions'][x_ind]
    dy = target['positions'][y_ind] - source['positions'][y_ind]
    distxy = math.sqrt(dx**2 + dy**2)
    prob = 1

    if con_pattern == Lamellar:
        prob = ratio/(math.exp(((abs(dx)-0)**2)/(2*(3**2))))
    if con_pattern == Homogenous:
        prob = ratio
    if con_pattern == IntPyrFeedback or con_pattern == AxonalPlexus:
        c = ratio
        a = gaussa
        prob = a /(math.exp(((abs(distxy)-0)**2)/(2*(c**2))))

    if random.random() < prob:
        #Since there will be recurrect connections we need to keep track externally to BMTK
        #BMTK will call build_edges twice if we use net.edges() before net.build()
        #Resulting in double the edge count
        syn_list.append({'source_gid':source['node_id'],'target_gid':target['node_id']})
        return random.randint(min_syn,max_syn)
    else:
        return 0

###########################################################
# Build recurrent connection rules
###########################################################
def hipp_recurrent_connector(source,target,all_edges=[],min_syn=1, max_syn=1):
    """
    General logic:
    1. Given a *potential* source and target
    2. Look through all edges currently made
    3. If any of the current edges contains 
        a. the current source as a previous target of 
        b. the current target as a prevous source
    4. Return number of synapses per this connection, 0 otherwise (no connection)
    """
    for e in all_edges:
        #if source['node_id'] == e.target_gid and target['node_id'] == e.source_gid:
        if source['node_id'] == e['target_gid'] and target['node_id'] == e['source_gid']:
            return random.randint(min_syn,max_syn)

    return 0

def syn_dist_delay(source, target, base_delay, dist_delay=None):#, min_weight, max_weight):
    """
    Original Code:
    distDelay = 0.1* (0.5*dist + rC.normal(0,1.5)*(1-exp(-dist^2/3)) ) 
    """
    base_delay = float(base_delay)
    if dist_delay:
        dist_delay = float(dist_delay)

    if dist_delay: #An override of sorts
        return base_delay + dist_delay

    x_ind,y_ind,z_ind = 0,1,2

    dx = target['positions'][x_ind] - source['positions'][x_ind]
    dy = target['positions'][y_ind] - source['positions'][y_ind]
    dz = target['positions'][z_ind] - source['positions'][z_ind]

    dist = math.sqrt(dx**2 + dy**2 + dz**2)
    distDelay = 0.1* (0.5*dist + np.random.normal(0,1.5,1)[0]*(1-math.exp(-dist**2/3)) ) 
    return float(base_delay) + distDelay

def syn_dist_delay_section(source, target, base_delay, dist_delay=None, sec_id=0, sec_x=0.9):
    return syn_dist_delay(source, target, base_delay, dist_delay), sec_id, sec_x


This first two parameters of this function is "source" and "target" and are required for all custom connector functions. These are node objects which gives a representation of a single source and target cell, with properties that can be accessed like a python dictionary. When The Network Builder is creating the connection matrix, it will call this function for all possible source-target pairs. The user doesn't call this function directly.

The remaining parameters are optional. Using these parameters, plus the distance and angles between source and target cells, this function determines the number of connections between each given source and target cell. If there are none you can return either None or 0.

To create these connections we call add_edges method of the builder. We use the source and target parameter to filter out only excitory-to-excitory connections. We must also take into consideration the model type (biophysical or integrate-and-fire) of the target when setting parameters. We pass in the function throught the connection_rule parameter, and the function parameters (except source and target) through connection_params. (If our dist_tuning_connector function didn't have any parameters other than source and target, we could just not set connection_params).

In [5]:
dynamics_file = 'CA3o2CA3e.inh.json'

conn = net.add_edges(source={'pop_name': 'CA3o'}, target={'pop_name': 'CA3e'},
            connection_rule=hipp_dist_connector,
            connection_params={'con_pattern':syn[dynamics_file]['con_pattern'],
                            'ratio':syn[dynamics_file]['ratio'],
                            'gaussa':syn[dynamics_file]['gaussa']},
            syn_weight=1,
            dynamics_params=dynamics_file,
            model_template=syn[dynamics_file]['level_of_detail'],
            distance_range=[0.0, 300.0],
            target_sections=['soma'])
conn.add_properties(names=['delay', 'sec_id', 'sec_x'],
            rule=syn_dist_delay_section,
            rule_params={'base_delay':syn[dynamics_file]['delay'], 'sec_id':0, 'sec_x':0.9},
            dtypes=[np.float, np.int32, np.float])


Similarly we create the other types of connections. But since either the source, target, or both cells will not have the tuning_angle parameter, we don't want to use dist_tuning_connector. Instead we can use the built-in distance_connector function which just creates connections determined by distance.

In [6]:
dynamics_file = 'CA3e2CA3o.exc.json'

experiment = 'original'
if experiment == "SFN19-D": #Weight of the synapses are set to 6 from max weight of 2
    dynamics_file = 'CA3e2CA3o.exc.sfn19exp2d.json'

conn = net.add_edges(source={'pop_name': 'CA3e'}, target={'pop_name': 'CA3o'},
            connection_rule=hipp_recurrent_connector,
            connection_params={'all_edges':syn_list}, #net.edges()},
            syn_weight=1,
            dynamics_params=dynamics_file,
            model_template=syn[dynamics_file]['level_of_detail'],
            distance_range=[0.0, 300.0],
            target_sections=['soma'])
conn.add_properties(names=['delay', 'sec_id', 'sec_x'],
            rule=syn_dist_delay_section,
            rule_params={'base_delay':syn[dynamics_file]['delay'], 'dist_delay':0.1, 'sec_id':0, 'sec_x':0.9}, #Connect.hoc:274 0.1 dist delay
            dtypes=[np.float, np.int32, np.float])


Finally we build the network (this may take a bit of time since it's essentially iterating over all 400x400 possible connection combinations), and save the nodes and edges.

In [7]:
net.build()
net.save_nodes(output_dir='sim_theta/network')
net.save_edges(output_dir='sim_theta/network')

### Building external network

Next we want to create an external network consisting of virtual cells that form a feedforward network onto our V1, which will provide input during the simulation. We will call this LGN, since the LGN is the primary input the layer 4 cells of the V1 (if we wanted to we could also create multiple external networks and run simulations on any number of them). 

First we build our LGN nodes. Then we must import the V1 network nodes, and create connections between LGN --> V1.

In [8]:
from bmtk.builder.networks import NetworkBuilder

exp0net = NetworkBuilder('exp0net')

exp0net.add_nodes(N=CA3eTotal, model_type='virtual', pop_name='bgnoisevirtCA3', pop_group='bgnoisevirtCA3')


As before, we will use a customized function to determine the number of connections between each source and target pair, however this time our connection_rule is a bit different

In the previous example, our connection_rule function's first two arguments were the presynaptic and postsynaptic cells, which allowed us to choose how many synaptic connections between the pairs existed based on individual properties:
```python
def connection_fnc(source, target, ...):
    source['param'] # presynaptic cell params
    target['param'] # postsynaptic cell params
    ...
    return nsyns # number of connections between pair
```

But for our LGN --> V1 connection, we do things a bit differently. We want to make sure that for every source cell, there are a limited number of presynaptic targets. This is a not really possible with a function that iterates on a one-to-one basis. So instead we have a connector function who's first parameter is a list of all N source cell, and the second parameter is a single target cell. We return an array of integers, size N; which each index representing the number of synaptics between sources and the target. 

To tell the builder to use this schema, we must set iterator='all_to_one' in the add_edges method. (By default this is set to 'one_to_one'. You can also use 'one_to_all' iterator which will pass in a single source and all possible targets).

In [9]:
def target_ind_equals_source_ind(source, targets, offset=0, min_syn=1,max_syn=1):
    # Creates a 1 to 1 mapping between source and destination nodes
    total_targets = len(targets)
    syns = np.zeros(total_targets)
    target_index = source['node_id']
    syns[target_index-offset] = 1
    return syns
        

conn = exp0net.add_edges(target=net.nodes(pop_name='CA3e'),
            source={'pop_name':'bgnoisevirtCA3'},
            iterator='one_to_all',
            connection_rule=target_ind_equals_source_ind,
            connection_params={'offset':0},
            dynamics_params='NetCon2EC.exc.json',
            model_template='pyr2pyr',
            delay=0,
            syn_weight=1,
            )
conn.add_properties(['sec_id','sec_x'],rule=(0, 0.9), dtypes=[np.int32,np.float])


exp0net.build()
exp0net.save_nodes(output_dir='sim_theta/network')
exp0net.save_edges(output_dir='sim_theta/network')

## 2. Setting up BioNet

#### file structure.

Before running a simulation, we will need to create the runtime environment, including parameter files, run-script and configuration files. You can copy the files from an existing simuatlion, execute the following command:

```bash
$ python -m bmtk.utils.sim_setup  \
   --report-vars v                \
   --report-nodes 10,80           \
   --network sim_theta/network     \
   --dt 0.1                       \
   --tstop 3000.0                 \  
   --include-examples             \
   --compile-mechanisms           \ 
   bionet sim_ch04
```

$ python -m bmtk.utils.sim_setup --report-vars v --report-nodes 0,80,100,300 --network sim_theta/network --dt 0.1 --tstop 3000.0  --include-examples --compile-mechanisms bionet sim_theta

or run it directly in python

In [10]:
from bmtk.utils.sim_setup import build_env_bionet

build_env_bionet(base_dir='sim_theta',      
                 network_dir='sim_theta/network',
                 tstop=3000.0, dt=0.1,
                 report_vars=['v'],     # Record membrane potential (default soma)
                 include_examples=True,    # Copies components files
                 compile_mechanisms=True   # Will try to compile NEURON mechanisms
                )

ERROR:bmtk.utils.sim_setup:  Was unable to compile mechanism in C:\Users\Tyler\Desktop\git_stage\theta\sim_theta\components\mechanisms


This will fill out the **sim_ch04** with all the files we need to get started to run the simulation. Of interest includes

* **circuit_config.json** - A configuration file that contains the location of the network files we created above. Plus location of neuron and synpatic models, templates, morphologies and mechanisms required to build our instantiate individual cell models.


* **simulation_config.json** - contains information about the simulation. Including initial conditions and run-time configuration (_run_ and _conditions_). In the _inputs_ section we define what external sources we will use to drive the network (in this case a current clamp). And in the _reports_ section we define the variables (soma membrane potential and calcium) that will be recorded during the simulation 


* **run_bionent.py** - A script for running our simulation. Usually this file doesn't need to be modified.


* **components/biophysical_neuron_models/** - The parameter file for the cells we're modeling. Originally [downloaded from the Allen Cell Types Database](http://celltypes.brain-map.org/neuronal_model/download/482934212). These files were automatically copies over when we used the _include-examples_ directive. If using a differrent or extended set of cell models place them here


* **components/biophysical_neuron_models/** - The morphology file for our cells. Originally [downloaded from the Allen Cell Types Database](http://celltypes.brain-map.org/neuronal_model/download/482934212) and copied over using the _include_examples_.


* **components/point_neuron_models/** - The parameter file for our LIF_exc and LIF_inh cells.


* **components/synaptic_models/** - Parameter files used to create different types of synapses.


#### lgn input

We need to provide our LGN external network cells with spike-trains so they can activate our recurrent network. Previously we showed how to do this by generating csv files. We can also use NWB files, which are a common format for saving electrophysiological data in neuroscience.

We can use any NWB file generated experimentally or computationally, but for this example we will use a preexsting one. First download the file:
```bash
 $ wget https://github.com/AllenInstitute/bmtk/blob/develop/docs/examples/spikes_inputs/lgn_spikes.nwb?raw=true
```
or copy from [here](https://github.com/AllenInstitute/bmtk/tree/develop/docs/examples/spikes_inputs/lgn_spikes.nwb).


Then we must edit the **simulation_config.json** file to tell the simulator to find the nwb file and which network to associate it with.

```json
{
  "inputs": {
    "LGN_spikes": {
      "input_type": "spikes",
      "module": "nwb",
      "input_file": "$BASE_DIR/lgn_spikes.nwb",
      "node_set": "LGN",
      "trial": "trial_0"
    }
  }
}
```


## 3. Running the simulation


We are close to running our simulation, however unlike in previous chapters we need a little more programming before we can begin. 

For most of the connections we added the parameter weight_function='wmax'. This is a built-in function that tells the simulator when creating a connection between two cells, just use the 'weight_max' value assigned to that given edge-type. 

However, when creating excitatory-to-excitatory connections we used weight_function='gaussianLL'. This is because we want to use the tuning_angle parameter, when avaiable, to determine the synaptic strength between two connections. First we create the function which takes in target, source and connection properties (which are just the edge-type and properties set in the add_edges method). Then we must register the function with the BioNet simulator:

In [11]:
import math
from bmtk.simulator.bionet.pyfunction_cache import add_weight_function

def gaussianLL(edge_props, source, target):
    src_tuning = source['tuning_angle']
    tar_tuning = target['tuning_angle']
    w0 = edge_props["syn_weight"]
    sigma = edge_props["weight_sigma"]

    delta_tuning = abs(abs(abs(180.0 - abs(float(tar_tuning) - float(src_tuning)) % 360.0) - 90.0) - 90.0)
    return w0 * math.exp(-(delta_tuning / sigma) ** 2)

add_weight_function(gaussianLL)

The weights will be adjusted before each simulation, and the function can be changed between different runs.. Simply opening the edge_types.csv file with a text editor and altering the weight_function column allows users to take an existing network and readjust weights on-the-fly.

Finally we are ready to run the simulation. Note that because this is a 400 cell simulation, this may be computationally intensive for some older computers and may take anywhere between a few minutes to half-an-hour to complete.

In [12]:
from bmtk.simulator import bionet


conf = bionet.Config.from_json('sim_theta/simulation_config.json')
conf.build_env()
net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
sim.run()

2020-09-28 22:46:28,632 [INFO] Created log file


INFO:NEURONIOUtils:Created log file


2020-09-28 22:46:28,855 [INFO] Building cells.


INFO:NEURONIOUtils:Building cells.


2020-09-28 22:46:29,042 [INFO] Building recurrent connections


...

INFO:NEURONIOUtils:    step:25000 t_sim:2500.00 ms


2020-09-28 22:49:54,258 [INFO]     step:30000 t_sim:3000.00 ms


INFO:NEURONIOUtils:    step:30000 t_sim:3000.00 ms


2020-09-28 22:49:54,278 [INFO] Simulation completed in 3.0 minutes, 24.28 seconds 


INFO:NEURONIOUtils:Simulation completed in 3.0 minutes, 24.28 seconds 


## 4. Analyzing results

Results of the simulation, as specified in the config, are saved into the output directory. Using the analyzer functions, we can do things like plot the raster plot

In [13]:
from bmtk.analyzer.spike_trains import plot_raster, plot_rates_boxplot

plot_raster(config_file='sim_theta/simulation_config.json', group_by='pop_name')

c:\users\tyler\desktop\git_stage\bmtk\bmtk\simulator\utils\config.py:4: UserWarning: Please use bmtk.simulator.core.simulation_config instead.
  warnings.warn('Please use bmtk.simulator.core.simulation_config instead.')


IndexError: list index out of range

and the rates of each node

In [ ]:
plot_rates_boxplot(config_file='sim_ch04/simulation_config.json', group_by='pop_name')

In our config file we used the cell_vars and node_id_selections parameters to save the calcium influx and membrane potential of selected cells. We can also use the analyzer to display these traces:

In [ ]:
from bmtk.analyzer.compartment import plot_traces

_ = plot_traces(config_file='sim_ch04/simulation_config.json', group_by='pop_name', report_name='v_report')